## user agent 설정하기
* 특정 사이트는 스크래핑이나 크롤링을 막기 위한 방편으로
* 사이트에 접속하는 사용자의 user agent를 확인함
* UA없이 사이트 접속을 시도하면 접속권한의 의미로 403 응답코드와 함께 접속을 금지하기도 함

In [2]:
import requests
from lxml import html

### JTBC 뉴스 속보 사이트에서 기사 크롤링하기
* 제목, 기사미리보기, 카테고리, 기자명, 송고날짜

In [3]:
url = 'https://news.jtbc.co.kr/section/list.aspx?scode='
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'}

### 크롤링 시작

In [34]:
res = requests.get(url, headers=headers)
text = html.fromstring(res.text)

### 추출한 요소들을 리스트에 저장 

In [35]:
titles, previews, categorys, reporters, pdates = [], [], [], [], []

# 제목 추출  
for title in doctree.cssselect('dt.title_cr a'):
    # print(title.text_content())
    titles.append(title.text_content())
# 미리보기 추출  
for preview in doctree.cssselect('dd.read_cr a'):
    previews.append(preview.text_content())

In [36]:
# 카테고리 추출  
for category in doctree.cssselect('span.location'):
    cate = category.text_content().strip()
    cate = cate.replace('\r','').replace('\n','')
    cate = cate.replace('\t','').replace(' ','')
    cate = cate.replace('[JTBC>','').replace(']','')
    categorys.append(cate)

In [37]:
# 기자명 추출  
for reporter in text.cssselect('span.writer'):
    # print(reporter.text_content().strip())
    reporters.append(reporter.text_content().strip())

In [55]:
# 송고날짜 추출  
for pdate in text.cssselect('span.date'):
    # print(pdate.text_content())
    pdates.append(pdate.text_content().strip())

In [39]:
print(len(titles),len(previews),len(categorys),len(reporters),len(pdates))

15 15 15 15 15


### 리스트에 저장한 내용을 파일에 저장

In [56]:
import json
from collections import OrderedDict

In [57]:
jtbcnews = OrderedDict()
items = OrderedDict()
body = OrderedDict()
news = []

for i in range(len(titles)):
    item = OrderedDict()
    item['title'] = titles[i]
    item['preview'] = previews[i]
    item['category'] = categorys[i]
    item['reporter'] = reporters[i]
    item['pdate'] = pdates[i]
    news.append(item)

items['items'] = news
body['body'] = items
jtbcnews['response'] = body

In [58]:
with open('./data/jtbcnews.json', 'w', encoding='utf-8') as f:
       json.dump(jtbcnews, f, ensure_ascii=False)

### 리스트에 저장한 내용을 디비에 저장

In [ ]:
# jtbcnews 테이블 구조 : 제목, 기사미리보기, 카테고리, 기자명, 송고날짜
# create table jtbcnews (
#     jtbcno     int            auto_increment,
#     title      varchar(100)   not null,
#     preview    varchar(350)   not null,
#     category   varchar(50)    not null,
#     reporter    varchar(50)    not null,
#     pdate      char(25)       not null,
#     primary key (jtbcno)
# );

In [59]:
import mariadb

In [60]:
# mariadb 접속 정보
url = 'bigdata.cto1qluvf6xh.ap-northeast-2.rds.amazonaws.com'
uid = 'admin'
pwd = 'Bigdata_2023'
db = 'bigdata'

In [62]:
sql = 'insert into jtbcnews (title, preview, category, reporter, pdate) values (?,?,?,?,?)'
conn = None
cur = None

try:
    conn = mariadb.connect(host=url, user=uid, password=pwd, database=db)
    cur = conn.cursor()

    for i in range(len(titles)):
        params = (titles[i], previews[i], categorys[i], reporters[i], pdates[i])
        cur.execute(sql, params)
        conn.commit()
        print(f'{i+1}번째 데이터 입력됨!')

except mariadb.Error as e:
    print('오류발생!!', e)
finally:
    cur.close()
    conn.close()

1번째 데이터 입력됨!
2번째 데이터 입력됨!
3번째 데이터 입력됨!
4번째 데이터 입력됨!
5번째 데이터 입력됨!
6번째 데이터 입력됨!
7번째 데이터 입력됨!
8번째 데이터 입력됨!
9번째 데이터 입력됨!
10번째 데이터 입력됨!
11번째 데이터 입력됨!
12번째 데이터 입력됨!
13번째 데이터 입력됨!
14번째 데이터 입력됨!
15번째 데이터 입력됨!
